# Imports

In [1]:
import requests
import pandas as pd

# GETs

In [2]:
base_url = 'https://pokeapi.co/api/v2/'

## Empty variables

In [3]:
# Define a list of cursor characters
cursor = ['-', '*']

# Create empty variables to store 'for' data
pokemon_df = pd.DataFrame()
pokemon_moves_array = []
pokemon_abilities_array = []
moves_df = pd.DataFrame()
abilities_df = pd.DataFrame()

### Pokemon

In [4]:
# The offset and limit parameters for pagination
offset = 0
limit = 1000

# A list to store all the Pokemon data
pokemon_list = []

# Loop through the pages of the API and retrieve the data
while True:
    # Make a GET request to the API with the offset and limit parameters
    response = requests.get(base_url+'pokemon', params={'offset': offset, 'limit': limit})
    # Check if the response was successful
    if response.status_code != 200:
        # If not, print an error message and break out of the loop
        print(f'Error: {response.status_code}')
        break
    # If the response was successful, retrieve the JSON data
    data = response.json()
    # Add the Pokemon data to the list
    pokemon_list.extend(data['results'])
    # Check if there are more pages of data to retrieve
    if not data['next']:
        # If not, break out of the loop
        break
    # If there are more pages, update the offset parameter
    offset += limit

max_size = len(pokemon_list)

# Drop useless Pokemon
for pokemon in pokemon_list[1009:]:
    pokemon_list.remove(pokemon)
    
dropped = max_size - len(pokemon_list)

print(f'Retrieved {max_size} Pokemon')
print(f'Dropped from list {dropped} Pokemon')

Retrieved 1281 Pokemon
Dropped from list 272 Pokemon


In [5]:
# Fetch Pokemon data from each url and store on dataframes
for pokemon in pokemon_list:
    response = requests.get(pokemon['url'])
    pokemon_data = response.json()
    
    base_stats = []
    
    for stat in pokemon_data['stats']:
        base_stats.append(stat['base_stat'])
    
    types = pokemon_data['types']
    
    if(len(types) > 1):
        types = [
            types[0]['type']['name'], 
            types[1]['type']['name']
        ]
    else:
        types = [
            types[0]['type']['name'],
            "NaN"
        ]
    
    df = pd.DataFrame({
        'name': [pokemon_data['name']],
        'hp': [base_stats[0]],
        'attack': [base_stats[1]],
        'defense': [base_stats[2]],
        'special-atk': [base_stats[3]],
        'special-defense': [base_stats[4]],
        'speed': [base_stats[5]],
        'type-1': [types[0]],
        'type-2': [types[1]],
    })
    
    abilities = [pokemon_data['name']]
    moves = [pokemon_data['name']]
    
    for ability in pokemon_data['abilities']:
        abilities.append(ability['ability']['name'])
    
    for move in pokemon_data['moves']:
        moves.append(move['move']['name'])
    
    for c in cursor:
        print(f"\rLoading... Iteration {pokemon_data['id']} of 1009: {c}", end='')
    
    pokemon_df = pd.concat([pokemon_df, df], ignore_index=True)
    pokemon_moves_array.append(moves)
    pokemon_abilities_array.append(abilities)

pokemon_moves_df = pd.DataFrame(pokemon_moves_array)
pokemon_abilities_df = pd.DataFrame(pokemon_abilities_array)

Loading... Iteration 1009 of 1009: *

### Moves

In [6]:
offset = 0
moves_list = []

# Loop through the pages of the API and retrieve the data
while True:
    # Make a GET request to the API with the offset and limit parameters
    response = requests.get(base_url+'move', params={'offset': offset, 'limit': limit})
    # Check if the response was successful
    if response.status_code != 200:
        # If not, print an error message and break out of the loop
        print(f'Error: {response.status_code}')
        break
    # If the response was successful, retrieve the JSON data
    data = response.json()
    # Add the Pokemon data to the list
    moves_list.extend(data['results'])
    # Check if there are more pages of data to retrieve
    if not data['next']:
        # If not, break out of the loop
        break
    # If there are more pages, update the offset parameter
    offset += limit

print(f'Retrieved {len(moves_list)} moves')

Retrieved 920 moves


In [7]:
# Fetch moves data and store to dataframe
for move in moves_list:
    response = requests.get(move['url'])
    move_data = response.json()
    
    effect_chance = move_data['effect_chance']
    effect_entry = ''
    
    if(effect_chance == None):
        effect_chance = 'NaN'
    
    if(len(move_data['effect_entries']) > 0):
        effect_entry = move_data['effect_entries'][0]['effect']
    
    move_list = [
        [move_data['name']],
        [move_data['accuracy']],
        [effect_chance],
        [move_data['pp']],
        [move_data['priority']],
        [move_data['power']],
        [move_data['damage_class']['name']],
        [effect_entry],
        [move_data['target']['name']],
        [move_data['type']['name']]
    ]
    
    df = pd.DataFrame({
        'name': move_list[0],
        'accuracy': move_list[1],
        'effect_chance': move_list[2],
        'pp': move_list[3],
        'priority': move_list[4],
        'power': move_list[5],
        'damage_class': move_list[6],
        'effect_text': move_list[7],
        'target': move_list[8],
        'type': move_list[9]
    })
    
    for c in cursor:
        print(f"\rLoading... Iteration {move_data['id']} of 920: {c}", end='')
    
    moves_df = pd.concat([moves_df, df], ignore_index=True)

Loading... Iteration 10018 of 920: *

### Abilities

In [8]:
offset = 0
abilities_list = []

# Loop through the pages of the API and retrieve the data
while True:
    # Make a GET request to the API with the offset and limit parameters
    response = requests.get(base_url+'ability', params={'offset': offset, 'limit': limit})
    # Check if the response was successful
    if response.status_code != 200:
        # If not, print an error message and break out of the loop
        print(f'Error: {response.status_code}')
        break
    # If the response was successful, retrieve the JSON data
    data = response.json()
    # Add the Pokemon data to the list
    abilities_list.extend(data['results'])
    # Check if there are more pages of data to retrieve
    if not data['next']:
        # If not, break out of the loop
        break
    # If there are more pages, update the offset parameter
    offset += limit
print(f'Retrieved {len(abilities_list)} abilities')

Retrieved 358 abilities


In [9]:
# Fetch abilities data and store to dataframe
for ability in abilities_list:
    response = requests.get(ability['url'])
    ability_data = response.json()
    
    effect_entry = ''
    flavor_text = ''
    effect_name = ability_data['name']
    
    for effect in ability_data['effect_entries']:
        entry = effect['effect']
        if(effect['language']['name'] == 'en'):
            effect_entry = 'entry'
            
    for effect in ability_data['flavor_text_entries']:
        entry = effect['flavor_text']
        if(effect['language']['name'] == 'en'):
            if(effect['version_group']['name'] == 'sword-shield'):
                flavor_text = 'entry'
    
    df = pd.DataFrame({
        'name': [effect_entry],
        'effect': [effect_name],
        'flavor_text': [flavor_text]
    })
    
    for c in cursor:
        print(f"\rLoading... Iteration {ability_data['id']} of 358: {c}", end='')
    
    abilities_df = pd.concat([abilities_df, df], ignore_index=True)

Loading... Iteration 10060 of 358: *

In [10]:
print('Pokemon df')
pokemon_df.head()
print('Pokemon abilities df')
pokemon_abilities_df.head()
print('Pokemon moves df')
pokemon_moves_df.head()
print('Moves df')
moves_df.head()
print('Abilities df')
abilities_df.head()

Pokemon df
Pokemon abilities df
Pokemon moves df
Moves df
Abilities df


,name,effect,flavor_text
0,entry,stench,entry
1,entry,drizzle,entry
2,entry,speed-boost,entry
3,entry,battle-armor,entry
4,entry,sturdy,entry


In [11]:
pokemon_df.to_csv('Pokemon/pokemon.csv')
pokemon_abilities_df.to_csv('Pokemon/pokemon_abilities.csv')
pokemon_moves_df.to_csv('Pokemon/pokemon_moves.csv')
moves_df.to_csv('Pokemon/moves.csv')
abilities_df.to_csv('Pokemon/abilities.csv')